# Course CSV → iOS Calendar (.ics)

This notebook converts a CSV timetable to an iOS-compatible `.ics` calendar file.

Supports:
- Weekly or biweekly recurrences within custom date ranges
- Single-date classes
- Exceptions (skip dates) and additional dates (make-up classes)

CSV columns (you can mix recurring rows and single-date rows):
- Recurring row: `name, weekday, start, end, location, start_date, end_date, count, interval, exceptions, rdates`
  - Required: `name, weekday, start, end, start_date`
  - One of `end_date` or `count` required; `interval` default 1 (weekly), set to 2 for biweekly
  - Optional: `location, exceptions (YYYY-MM-DD, ...), rdates (YYYY-MM-DD, ...)`
- Single-date row: `name, date, start, end, location`

Examples:
```csv
name,weekday,start,end,location,start_date,end_date,count,interval,exceptions,rdates,date
ACCT7101 - Accounting Theory,Mon,19:00,22:00,KKL-LG.1,2025-09-01,2025-09-30,,1,2025-09-29,,
ECON7202 - Microeconomics,Tue,14:00,17:00,CPD-3.04,2025-09-01,,4,1,,,
ECON7202 - Microeconomics extra,,,,CPD-3.04,,,,,,,2025-09-20,14:00,17:00
```

After setting parameters below, run the cells in order to generate the `.ics` file.


In [ ]:
# Parameters
CSV_PATH = "/Users/username/Desktop/TimeTable.csv"  # Update if needed
OUTPUT_ICS = "/Users/username/Desktop/hku_course_scraper/hku_courses.ipynb.ics"
TIMEZONE = "Asia/Hong_Kong"  # IANA TZ



In [ ]:
# Install deps if needed (run once)
import sys, subprocess

def ensure(package: str):
    try:
        __import__(package)
    except Exception:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

for pkg in ["icalendar"]:
    ensure(pkg)
print("OK")



In [ ]:
import csv
import dataclasses
import datetime as dt
import re
from pathlib import Path
from typing import Dict, Optional

from icalendar import Calendar, Event


WEEKDAY_ALIASES = {
    "mon": "MO", "monday": "MO", "1": "MO",
    "tue": "TU", "tues": "TU", "tuesday": "TU", "2": "TU",
    "wed": "WE", "weds": "WE", "wednesday": "WE", "3": "WE",
    "thu": "TH", "thur": "TH", "thurs": "TH", "thursday": "TH", "4": "TH",
    "fri": "FR", "friday": "FR", "5": "FR",
    "sat": "SA", "saturday": "SA", "6": "SA",
    "sun": "SU", "sunday": "SU", "7": "SU",
    "周一": "MO", "周二": "TU", "周三": "WE", "周四": "TH", "周五": "FR", "周六": "SA", "周日": "SU",
}

def normalize_weekday(text: str) -> Optional[str]:
    t = re.sub(r"\s+", " ", (text or "")).strip().lower()
    return WEEKDAY_ALIASES.get(t)

def parse_hhmm(text: str) -> dt.time:
    m = re.fullmatch(r"(\d{1,2}):(\d{2})", text.strip())
    if not m:
        raise ValueError(f"Invalid time: {text}")
    h, mnt = map(int, m.groups())
    return dt.time(h, mnt)

def parse_date(text: str) -> dt.date:
    s = (text or "").strip().replace(".", "-").replace("/", "-")
    m = re.fullmatch(r"(\d{4})-(\d{1,2})-(\d{1,2})", s)
    if m:
        y, mo, d = map(int, m.groups())
        return dt.date(y, mo, d)
    m = re.fullmatch(r"(\d{1,2})-(\d{1,2})-(\d{4})", s)
    if m:
        d, mo, y = map(int, m.groups())
        return dt.date(y, mo, d)
    return dt.date.fromisoformat(s)

def get_tz(tz_name: str):
    try:
        from zoneinfo import ZoneInfo
        return ZoneInfo(tz_name)
    except Exception:
        import pytz
        return pytz.timezone(tz_name)

def first_date_for_weekday(start_date: dt.date, weekday_code: str) -> dt.date:
    target = ["MO", "TU", "WE", "TH", "FR", "SA", "SU"].index(weekday_code)
    offset = (target - start_date.weekday()) % 7
    return start_date + dt.timedelta(days=offset)

def add_exdates(event: Event, tz, exdates_csv: str) -> None:
    for part in (exdates_csv or "").split(','):
        part = part.strip()
        if not part:
            continue
        d = parse_date(part)
        event.add('exdate', dt.datetime.combine(d, dt.time.min).replace(tzinfo=tz))

def add_rdates(event: Event, tz, rdates_csv: str, start_time: dt.time) -> None:
    for part in (rdates_csv or "").split(','):
        part = part.strip()
        if not part:
            continue
        d = parse_date(part)
        event.add('rdate', dt.datetime.combine(d, start_time).replace(tzinfo=tz))

def create_event_recurring(row: Dict[str, str], tz_name: str) -> Event:
    tz = get_tz(tz_name)
    name = (row.get('name') or '').strip() or 'Course'
    location = (row.get('location') or '').strip()
    weekday = normalize_weekday(row.get('weekday', ''))
    if not weekday:
        raise SystemExit(f"Invalid weekday: {row.get('weekday')} for {name}")
    start = parse_hhmm(row['start'])
    end = parse_hhmm(row['end'])
    start_date = parse_date(row['start_date'])
    dtstart_date = first_date_for_weekday(start_date, weekday)
    dtstart = dt.datetime.combine(dtstart_date, start).replace(tzinfo=tz)
    dtend = dt.datetime.combine(dtstart_date, end).replace(tzinfo=tz)

    ev = Event()
    ev.add('summary', name)
    if location:
        ev.add('location', location)
    ev.add('dtstart', dtstart)
    ev.add('dtend', dtend)

    interval = int((row.get('interval') or '1').strip() or '1')
    rrule: Dict[str, object] = {'freq': 'weekly', 'interval': interval}
    count_text = (row.get('count') or '').strip()
    end_date_text = (row.get('end_date') or '').strip()
    if count_text:
        rrule['count'] = int(count_text)
    elif end_date_text:
        until_date = parse_date(end_date_text)
        until_dt = dt.datetime.combine(until_date, dt.time(23, 59)).replace(tzinfo=tz)
        rrule['until'] = until_dt
    else:
        rrule['count'] = 30

    ev.add('rrule', rrule)
    add_exdates(ev, tz, row.get('exceptions', ''))
    add_rdates(ev, tz, row.get('rdates', ''), start)
    return ev

def create_event_single(row: Dict[str, str], tz_name: str) -> Event:
    tz = get_tz(tz_name)
    name = (row.get('name') or '').strip() or 'Course'
    location = (row.get('location') or '').strip()
    start = parse_hhmm(row['start'])
    end = parse_hhmm(row['end'])
    date = parse_date(row['date'])
    dtstart = dt.datetime.combine(date, start).replace(tzinfo=tz)
    dtend = dt.datetime.combine(date, end).replace(tzinfo=tz)

    ev = Event()
    ev.add('summary', name)
    if location:
        ev.add('location', location)
    ev.add('dtstart', dtstart)
    ev.add('dtend', dtend)
    return ev

def build_calendar_from_csv(csv_path: Path, tz_name: str) -> Calendar:
    with csv_path.open('r', encoding='utf-8-sig', newline='') as f:
        reader = csv.DictReader(f)
        headers = {h.strip().lower() for h in (reader.fieldnames or [])}
        base = {'name', 'start', 'end'}
        missing = [h for h in base if h not in headers]
        if missing:
            raise SystemExit(f"CSV missing columns: {', '.join(missing)}. Always required: name,start,end")

        cal = Calendar()
        cal.add('prodid', '-//Notebook CSV Course Importer//iOS//')
        cal.add('version', '2.0')

        for row in reader:
            row_lc = {k.lower(): v for k, v in row.items()}
            has_date = bool(row_lc.get('date'))
            if has_date:
                ev = create_event_single(row_lc, TIMEZONE)
                cal.add_component(ev)
                continue
            for req in ('weekday', 'start_date'):
                if not row_lc.get(req):
                    raise SystemExit(f"Recurring row requires '{req}': {row}")
            ev = create_event_recurring(row_lc, TIMEZONE)
            cal.add_component(ev)

    return cal



In [ ]:
# Build calendar and write ICS
cal = build_calendar_from_csv(Path(CSV_PATH), TIMEZONE)
outfile = Path(OUTPUT_ICS)
outfile.write_bytes(cal.to_ical())
print(f"Wrote {outfile}")

